In [1]:
import json
import random
import malaya

2023-07-28 20:04:45.593890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 20:04:45.661330: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-28 20:04:46.197834: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-28 20:04:46.197880: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
# !pip3 uninstall malaya -y
# !pip3 install git+https://github.com/huseinzol05/malaya@5.1 --no-deps

In [3]:
language_detection = malaya.language_detection.fasttext(model = 'mesolitica/fasttext-language-detection-ms-id')

In [4]:
template_malaya = {
    'description': 'Template used by Malaya.',
    'prompt_input': 'Di bawah ialah arahan yang menerangkan tugasan, termasuk dengan input yang menyediakan konteks lanjut. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Input:\n{input}\n\n### Jawapan:\n',
    'prompt_no_input': 'Di bawah ialah arahan yang menerangkan tugasan. Tulis jawapan yang sesuai dengan arahan tersebut.\n\n### Arahan:\n{instruction}\n\n### Jawapan:\n',
    'response_split': '### Jawapan:',
}

In [6]:
language_detection.predict(['helo'])

['standard-malay']

In [7]:
model = malaya.augmentation.abstractive.huggingface(model = 'mesolitica/translation-t5-small-standard-bahasa-cased')

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [8]:
_ = model.cuda()

In [9]:
def generate_noisy(text):
    g = model.generate([text],to_lang = 'pasar ms',
              max_length = 512, do_sample=True, top_k=100, top_p=0.95, temperature=0.7,
              num_return_sequences=5)
    result = []
    for g_ in g:
        splitted = g_.split()
        splitted = [s for s in splitted if s[0] !='@']
        s = ' '.join(splitted)
        if len(s):
            result.append(s)
    return result

generate_noisy('bagaimana saya boleh kurangkan berat badan tanpa pergi ke gim?')

['cam mana nak kurus tanpa pergi gym',
 'macam mana nak kurus tanpa pergi gym',
 'mcm mana nak kurus tanpa pergi gym',
 'Macam mana aku nak kurus tanpa pergi gym?',
 'macam mana nak kurus tanpa pergi gym ni']

In [16]:
from tqdm import tqdm

data = []
with open('/home/ubuntu/server2/ssd3/gpt4all-1.3/translated-gpt4all-filtered-noncode.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        try:
            a = l['r'][-1]['result']
            
            if language_detection.predict([a])[0] != 'standard-malay':
                continue
            
            ori_q = l['src']['prompt']
            
            if random.random() > 0.7:
                q = ori_q
            else:
                q = l['r'][0]['result']
                if language_detection.predict([q])[0] != 'standard-malay':
                    continue
                if random.random() > 0.5 and '|' not in ori_q and len(q.split()) < 20:
                    g = generate_noisy(q)
                    q = random.choice(g)
            
            o = {
                'prompt_input': None,
                'input': q,
                'output': a,
            }
            data.append(o)
            
        except Exception as e:
            pass

57908it [1:12:32,  7.54it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

254968it [4:33:50, 43.70it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

388519it [7:09:18, 12.79it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_li

In [17]:
with open('prepared-gpt4all-v1.3-noisy.jsonl', 'w') as fopen:
    for l in data:
        fopen.write(f'{json.dumps(l)}\n')

In [19]:
!head -n 5 prepared-gpt4all-v1.3-noisy.jsonl

{"prompt_input": null, "input": "Baca ini dan balas berdasarkan arahan saya.\n* Bandar Chongqing Lifan ialah Chongqing.\n* Stadium rumah Chongqing Lifan ialah Pusat Sukan Olimpik Chongqing.\n* Kehadiran Pusat Sukan Olimpik Chongqing ialah 11,433 orang.\n* Kapasiti Pusat Sukan Olimpik Chongqing ialah 58,680.", "output": "Bandar Chongqing Lifan ialah Chongqing, dan stadium rumah mereka ialah Pusat Sukan Olimpik Chongqing, yang mempunyai kapasiti 58,680. Bagaimanapun, kehadiran untuk satu acara baru-baru ini di stadium hanya 11,433, menunjukkan bahawa ia tidak dipenuhi sehingga kapasiti."}
{"prompt_input": null, "input": "Baca ini dan balas berdasarkan arahan saya.\n* Blog Chris Colfer ialah chriscolfer.\n* Watak Chris Colfer ialah Kurt Hummel.\n* Watak Chris Colfer ialah Ryan Kendall.\n* Filem Chris Colfer ialah 8.\n* Filem Chris Colfer ialah Glee Encore.\n* Filem Chris Colfer ialah Glee: The 3D Concert Movie.\n* Filem Chris Colfer ialah Russel Fish: The Sausage and Eggs Incident.\n* Fil